In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [ ]:
#import the dataset and selected the useful variables

data=pd.read_csv("../../data/exam_1_with_outcome_nses.csv")
select_exam_1=data[['idno','site1c','gender1','race1c','age1c','cig1c','chol1','hdl1','sbp1c','diabet1','nSES','F3_PC2','y']]
select_exam_1['race_2']=np.where(select_exam_1['race1c']==3,1,0)

#join the dataset with diagnosis day "fuptt"
data_event=pd.read_csv("../../data/outcomes/event.csv")
select_data_event=data_event[['idno','fuptt']]

event_exam_1=pd.merge(select_exam_1,select_data_event, how="left",on=['idno'])
event_exam_1.count()

In [ ]:
#step1: split the dataset to train and test (80/20) with stratified option

train_set_1, test_set_1=train_test_split(event_exam_1, 
                                         test_size=0.2, stratify=select_exam_1[['y','race_2','site1c']], random_state=1)
print(train_set_1.count())
print(test_set_1.count())

In [ ]:
#step2: filling in the missing data in train set

train_set_1['diabet1'] = train_set_1['diabet1'].replace(9,0)
train_set_1['diabet1'] = train_set_1['diabet1'].fillna(value=0)

train_set_1['cig1c'] = train_set_1['cig1c'].replace(2,1)
train_set_1['cig1c'] = train_set_1['cig1c'].fillna(value=0)



ave_hdl1 = train_set_1.mean()['hdl1']
train_set_1.loc[pd.isna(train_set_1['hdl1']),'hdl1'] = ave_hdl1

ave_chol1 = train_set_1.mean()['chol1']
train_set_1.loc[pd.isna(train_set_1['chol1']),'chol1'] = ave_chol1

ave_sbp1c = train_set_1.mean()['sbp1c']
train_set_1.loc[pd.isna(train_set_1['sbp1c']),'sbp1c'] = ave_sbp1c

ave_fuptt = train_set_1.mean()['fuptt']
train_set_1.loc[pd.isna(train_set_1['fuptt']),'fuptt'] = ave_fuptt


train_set_1 = train_set_1.drop(["race1c"], axis = 1)
print(train_set_1.count())


#filling in the missing data in test set using the average value calculated in train set
test_set_1['diabet1'] = test_set_1['diabet1'].replace(9,0)
test_set_1['diabet1'] = test_set_1['diabet1'].fillna(value=0)

test_set_1['cig1c'] = test_set_1['cig1c'].replace(2,1)
test_set_1['cig1c'] = test_set_1['cig1c'].fillna(value=0)

test_set_1.loc[pd.isna(test_set_1['hdl1']),'hdl1'] = ave_hdl1
test_set_1.loc[pd.isna(test_set_1['chol1']),'chol1'] = ave_chol1
test_set_1.loc[pd.isna(test_set_1['sbp1c']),'sbp1c'] = ave_sbp1c
test_set_1.loc[pd.isna(test_set_1['fuptt']),'fuptt'] = ave_fuptt


test_set_1 = test_set_1.drop(["race1c"], axis = 1)
print(test_set_1.count())


In [ ]:
#add interaction term to the dataset
train_set_1['gender_race'] = train_set_1['gender1']*train_set_1['race_2'] 
test_set_1['gender_race'] = test_set_1['gender1']*test_set_1['race_2'] 
print(train_set_1.count())
print(test_set_1.count())

In [ ]:
#standiazed

scaler_index = scaler.fit(train_set_1[['age1c','chol1','hdl1','sbp1c','F3_PC2']],)
scaler_index

df_index_tr = train_set_1[['age1c','chol1','hdl1','sbp1c','F3_PC2']].index
df_index_te = test_set_1[['age1c','chol1','hdl1','sbp1c','F3_PC2']].index
df_cols = train_set_1[['age1c','chol1','hdl1','sbp1c','F3_PC2']].columns

X_tr_con = scaler_index.transform(train_set_1[['age1c','chol1','hdl1','sbp1c','F3_PC2']], copy = True)
X_tr_con = pd.DataFrame(X_tr_con, index=df_index_tr, columns=df_cols)
X_te_con = scaler_index.transform(test_set_1[['age1c','chol1','hdl1','sbp1c','F3_PC2']], copy = True)
X_te_con = pd.DataFrame(X_te_con, index=df_index_te, columns=df_cols)

X_tr_cate = train_set_1.loc[:,~train_set_1.columns.isin(df_cols)]
X_te_cate = test_set_1.loc[:,~test_set_1.columns.isin(df_cols)]
X_train_scale = X_tr_cate.join(X_tr_con)
X_test_scale = X_te_cate.join(X_te_con)

In [ ]:
mesa_train_scale=pd.DataFrame(data=X_train_scale)
mesa_train_scale.to_csv("mesa_preprocess_train.csv",index=False,sep=',')
mesa_test_scale=pd.DataFrame(data=X_test_scale)
mesa_test_scale.to_csv("mesa_preprocess_test.csv",index=False,sep=',')